<a href="https://colab.research.google.com/github/cccadet/dermatologist-ai/blob/master/01_Test1_ISIC_2019_DeepStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pré: Conexão com Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Etapa 1: Instalação das dependências

In [ ]:
#!pip install tensorflow-gpu==2.1.0
!pip install tf-nightly-gpu
!pip install tqdm
!pip install kaggle
!pip install keras-rectified-adam
!pip install tensorflow-addons
#!pip install tfa-nightly

     |████████████████████████████████| 350.3MB 26kB/s 
     |████████████████████████████████| 460kB 53.6MB/s 
     |████████████████████████████████| 3.0MB 50.2MB/s 
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=453e4a19584402ec052bf523a7d74ed911627a628d8ea7f301aa9f75866536b9
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam


### Importação das bibliotecas

In [ ]:
import zipfile
import os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import pandas as pd
import sys
import itertools
from keras_radam import RAdam
import shutil

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

Using TensorFlow backend.


'2.3.0-dev20200616'

### Otimizações tensorflow

In [ ]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)
tf.config.optimizer.set_jit(True)

### Fazendo o download da base de dados de imagens

In [ ]:
%%time
if (os.path.isdir('./ISIC_2019_Test_Input')==False):
  !wget --no-check-certificate \
      https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip \
      -O ./skin_cancer_test.zip

--2020-06-17 01:57:57--  https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3823247963 (3.6G) [application/zip]
Saving to: ‘./skin_cancer_test.zip’

./skin_cancer_test. 100%[===================>]   3.56G  38.6MB/s    in 97s     

2020-06-17 01:59:34 (37.6 MB/s) - ‘./skin_cancer_test.zip’ saved [3823247963/3823247963]

CPU times: user 617 ms, sys: 199 ms, total: 816 ms
Wall time: 1min 38s


## Etapa 2: Pré-processamento

### Configurando os caminhos (paths)

In [ ]:
test_dir = "./test"

download_path = './'
#derm_ai_path_V1 = '/content/drive/My Drive/Colab Notebooks/Dermatologist_AI'
derm_ai_path = '/content/drive/My Drive/Derm AI/ISIC_2019'
#ground_truth_file = os.path.join(derm_ai_path, 'ground_truth.csv')

In [ ]:
def dcp(download_path, path, file):
  if (os.path.isdir(path)==False):
    dataset_path = os.path.join(download_path,file)
    zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
    zip_object.extractall("./")
    zip_object.close()

### Descompactando a base de dados de Treinamento

### Descompactando a base de dados de Testes

In [ ]:
%%time
dcp(download_path, './ISIC_2019_Test_Input', 'skin_cancer_test.zip')

CPU times: user 21.1 s, sys: 4.24 s, total: 25.3 s
Wall time: 1min 16s


In [ ]:
!rm ./skin_cancer_test.zip

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: InceptionResNetV2 (que estamos usando) suporta: (299x299).

In [ ]:
data_gen_test = ImageDataGenerator(rescale=1/255., 
                                   fill_mode="constant"
)

In [ ]:
!mkdir ./test
!mkdir ./test/test
!mv ./ISIC_2019_Test_Input/* ./test/test

In [ ]:
test_generator  = data_gen_test.flow_from_directory(test_dir, target_size=(299,299), batch_size=32, shuffle=False, class_mode=None)

Found 8238 images belonging to 1 classes.


#### Exibição de 3 imagens carregadas

In [ ]:
#x,y = test_generator.next()
#for i in range(0,3):
#    image = x[i]
#    label = y[i]
#    print (label)
#    plt.imshow(image)
#    plt.show()

## Etapa 3: Construindo o modelo

### Carregando o modelo pré-treinado (InceptionResNetV2)

### Congelando o modelo base

### Definindo o cabeçalho personalizado da rede neural

### Definindo o modelo

### Compilando o modelo

### Treinando o modelo

### Avaliação do modelo de transferência de aprendizagem

## Etapa 4: Fine tuning


Algumas questões principais:

- NEM SEMPRE é necessário realizar o Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 
- Mais informações em como usar o Fine Tuning acesse: 
 https://www.analyticsvidhya.com/blog/2017/06/transfer-learning-the-art-of-fine-tuning-a-pre-trained-model/

### Descongelando algumas camadas do topo do modelo base

Descomentar trecho abaixo para manter camadas congeladas. O número informado no parâmetro `fine_tuning_at` define o número das primeiras camadas que ficarão congeladas. Supondo que a rede tenha 780 camadas e seja informado o valor 500, até a camada 500 a rede ficará congelada.

### Compilando o modelo para fine tuning

### Fine tuning

### Avaliação do modelo com fine tuning

### Salvar modelo após Fine Tuning

In [ ]:
%%time
img_shape = (299, 299, 3)
base_model = tf.keras.applications.EfficientNetB5(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

#base_model.output
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
#global_average_layer
prediction_layer = tf.keras.layers.Dense(units = 8, activation = "softmax")(global_average_layer)

model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

radam = tfa.optimizers.RectifiedAdam(
    lr=1e-2,
    total_steps=634,
    warmup_proportion=0.1,
    min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

optm = ranger

model.compile(optimizer=optm, loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy()])

115269632/115263384 [==============================] - 2s 0us/step
CPU times: user 6.65 s, sys: 377 ms, total: 7.03 s
Wall time: 8.37 s


### Carregar modelo após Fine Tuning com os melhores pesos

In [ ]:
%%time

#InceptionResNetV2_model_file = os.path.join(derm_ai_path, 'InceptionResNetV2_full_cat.h5')
checkpoint_filepath = os.path.join(derm_ai_path, 'Efficientnet_B5_cat.weights.best.my.hdf5')

# Recreate the exact same model, including its weights and the optimizer
#model = tf.keras.models.load_model(InceptionResNetV2_model_file)
model.load_weights(checkpoint_filepath)


CPU times: user 531 ms, sys: 100 ms, total: 632 ms
Wall time: 704 ms


### Matriz de Validação

### Execução das predições na base de testes

In [ ]:
%%time

test_generator.reset()
pred=model.predict(test_generator,
                              verbose=1)

258/258 [==============================] - 155s 601ms/step
CPU times: user 3min 17s, sys: 17.6 s, total: 3min 34s
Wall time: 2min 57s


### Identificar classe predita

In [ ]:
#def predClasses(pred, train_generator):
#  predicted_class_indices=np.argmax(pred,axis=1)
#
#  labels = (train_generator.class_indices)
#  labels = dict((v,k) for k,v in labels.items())
#  predictions = [labels[k] for k in predicted_class_indices]
#  return predicted_class_indices, labels, predictions

#predicted_class_indices, labels, predictions = predClasses(pred, train_generator)

In [ ]:
#image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
#labels[5]
#predicted_class_indices[0]
#predictions[0]
#predicted_class_indices[0]
#pred[0].max()

In [ ]:
def is_unknown(pred, th):
  unk = []
  for row in pred:
    if row.max() < th:
        #Unknown class
        unk.append(1.0)
    else:
        unk.append(0.0)
  return unk

th = 0.35

### Salvar resultado para conferência

In [ ]:
pd.set_option('display.float_format', lambda x: '%.12f' % x)
results_file = os.path.join(derm_ai_path, 'results_efficientnet_b5.csv')

def saveResultsFile(results_file,pred, test_generator, unk):
  filenames=test_generator.filenames
  filenames = [w.replace('test/', '') for w in filenames]
  filenames = [w.replace('.jpg', '') for w in filenames]
  results=pd.DataFrame({"image":filenames,
                        "MEL":pred[:,4],
                        "NV":pred[:,5],
                        "BCC":pred[:,1],
                        "AK":pred[:,0],
                        "BKL":pred[:,2],
                        "DF":pred[:,3],
                        "VASC":pred[:,7],
                        "SCC":pred[:,6],
                        "UNK": unk
                        })
  results.to_csv(results_file,index=False, float_format='%.12f')

unk = is_unknown(pred, th)
saveResultsFile(results_file, pred, test_generator, unk)

In [ ]:
df_comp = pd.read_csv(results_file)
df_comp.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0034321,0.048717919737,0.375045388937,0.286978930235,0.005461120512,0.266873449087,0.009737982415,0.001952611725,0.005232505966,0.000000000000
1,ISIC_0034322,0.076323658228,0.920177400112,0.000200122595,0.000183712895,0.002513998654,0.000280109991,0.000088098837,0.000232879363,0.000000000000
2,ISIC_0034323,0.009104279801,0.006607472897,0.893086135387,0.016140082851,0.017394797876,0.004783717450,0.001154930447,0.051728587598,0.000000000000
3,ISIC_0034324,0.009652582929,0.984512686729,0.000352434872,0.000192541775,0.002299587009,0.001170233591,0.001720136963,0.000099743898,0.000000000000
4,ISIC_0034325,0.026937671006,0.967616856098,0.000714109861,0.000195745262,0.003325415310,0.000661650323,0.000366139284,0.000182484233,0.000000000000


### Rodar avaliação do modelo em relação a base de testes

### Salvar modelo com score obtido

### Carregar melhor modelo 

## Etapa 5: Tensorflow Lite

### Converter modelo para formato tflite

### Salvar arquivo convertido

### Testar modelo convertido (TFLite)
https://medium.com/towards-artificial-intelligence/testing-tensorflow-lite-image-classification-model-e9c0100d8de3

https://colab.research.google.com/github/frogermcs/TFLite-Tester/blob/master/notebooks/Testing_TFLite_model.ipynb#scrollTo=6WEyRJNsR5uL

https://aihub.cloud.google.com/p/products%2F6ca92c9c-6932-45f8-8f08-15a1a6dfafc7